In [63]:
import json
import re
import time

import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers

### ES Datebase에 넣을 Sample DATA-SET
    데이터 셋은 20180131 이전에 crawling한 데이터셋.
    - sample_item : items index에 해당하는 csv 파일
    - sample_review : reviews index에 해당하는 csv 파일

In [10]:
item_df = pd.read_csv("data/sample_item.csv",sep="▒",na_values="",engine='python')
review_df = pd.read_csv("data/sample_review.csv",sep='▒',na_values="",engine='python')

# review_tag는 json파일로 되어 있음. list 형태로 복원
review_df.review_tag = review_df.review_tag.apply(json.loads)

item_df = item_df.fillna("")
review_df = review_df.fillna("")

# item index에 review_tag 부착
item_df['review_tag'] = 0
item_df['review_tag'] = item_df.review_tag.apply(lambda x : ["가격", "디자인", "배송", "색", "조립", "품질"])

## Elasticsearch 관련 Script

In [11]:
item_index_name = "item_table"
review_index_name = "review_table"

#### 1. initialize ES-client 

In [64]:
es = Elasticsearch(host="localhost")
if es.ping():
    # Check Elasticsearch is operating
    print("Elasticsearch is Okay")

Elasticsearch is Okay


#### 2. check the existence of Index in ES

In [ ]:
delete_index = False # 인덱스 내용을 지우는 지 여부

if es.indices.exists(item_index_name):
    print('"items" index exists')
    if delete_index:
        es.indices.delete(item_index_name, ignore=[400,404])
if es.indices.exists(review_index_name):
    print('"reviews" index exists')
    if delete_index:
        es.indices.delete(review_index_name, ignore=[400,404])

#### 3. create the index with tokenizer setting

**stop word**
    
    자바 security manager의 관리 하에 있는 elasticsearch는 
    특정 파일을 읽으려면, 읽기 권한을 제공해주어야 한다.
    

````java
// path : /usr/lib/jvm/java-8-openjdk-amd64/jre/lib/security/java.policy
grant {
    permission java.io.FilePermission "/home/ubuntu/elasticsearch/stopword.csv", "read";
};
````
위와 같이 권한을 상속해야 한다.

**arirang analyzer**

    Tokenizer 내부에 문제가 있어서, 
    
    tartOffset must be non-negative, and endOffset must be >= startOffset, and offsets must not go backwards startOffset=0,endOffset=2,lastStartOffset=3 for field 'cat_1'"
    
    라는 내용의 offset error가 있음. 아래의 방식으로 해결해야함
    
**reference**
1. http://jjeong.tistory.com/1257
2. http://jjeong.tistory.com/1281

**은전한닢 토크나이저**
   
    은전한닢이 ES 내에서 가장 많이 쓰이는 토크나이저. 안정적이긴 한데 좀 메모리를 많이 먹는다는 평이 있음
   
1. http://blog.indexall.net/2017/05/installed-elasticsearch-analysis-mecab.html
2. https://bitbucket.org/eunjeon/seunjeon/raw/master/elasticsearch/

In [14]:
# reference : http://jjeong.tistory.com/1142
setting = {
    "settings":{
      "index.mapping.ignore_malformed": "true",
      "index":{
        "analysis":{
          "tokenizer": {
            "seunjeon_default_tokenizer": {
              "type": "seunjeon_tokenizer",
              "pos_tagging": False,
            },
            "word_cloud_tokenizer": {
              "type": "seunjeon_tokenizer",
              "index_eojeol" : False,
              "pos_tagging": False,
              "index_poses": ["N","V"]
            }
          },
          "analyzer":{
            # 은전한닢 토크나이저 세팅        
            "korean": {
              "type" : "custom",
              "tokenizer": "seunjeon_default_tokenizer",
            },
            "popular":{
              "tokenizer":"word_cloud_tokenizer",           
              "filter": ["stop_popular_word"]
            }
          },
          "filter":{
            "stop_popular_word" : {
                  "type" : "stop",
                  "stopwords_path" : "/home/ubuntu/elasticsearch/stopword.csv"
            }          
          }
        }
      }
    }
}

In [15]:
# create the index 
es.indices.create(item_index_name,body=json.dumps(setting))
es.indices.create(review_index_name,body=json.dumps(setting))

{'acknowledged': True, 'index': 'item_table', 'shards_acknowledged': True}

In [ ]:
# test  - korean
body = {
    "analyzer" :"popular",
    "text":review_df.sort_values('review_accuracy',ascending=False).iloc[10000].review
}
es.indices.analyze(item_index_name,body=json.dumps(body))

#### 4. Put mapping in Index

In [16]:
item_mapping = {
    "properties":{
        "brand_id"   : {"type":"integer"},
        "brand_name" : {"type":"keyword"},
        "img_url"    : {"type":"keyword"},
        "item_spec"  : {"type":"text", "analyzer":"korean"},
        "item_name"  : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}}, 
        "min_price"  : {"type":"long"},
        "nv_mid"     : {"type" : "long"},
        "url"  : {"type":"keyword"},
        "cat_1": {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_2": {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_3": {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_4": {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "review_tag" : {"type":"keyword"},
    }
}
es.indices.put_mapping("item",body=json.dumps(item_mapping),index=item_index_name)

{'acknowledged': True}

In [ ]:
review_mapping = {
    "properties":{
        "brand_id"   : {"type":"integer"},
        "brand_name" : {"type":"keyword"},
        "img_url"    : {"type":"keyword"},
        "item_spec"  : {"type":"text","analyzer":"korean"},
        "item_name"  : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "min_price"  : {"type":"long"},
        "nv_mid"     : {"type":"long"},
        "url"   : {"type":"keyword"},
        "cat_1" : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_2" : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_3" : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "cat_4" : {"type":"text","analyzer":"korean","fields":{"raw":{"type":"keyword"}}},
        "review": {"type":"text","analyzer":"korean",
                    "fields": {
                        "raw":{"type":"keyword"},
                        "tokens":{"type":"text","fielddata":"true","analyzer":"popular"}
                     },
                  },
        "review_date"  : {"type" : "date", "format":"yyyy.MM.||yyyy.MM||yyyy.MM.dd||yyyy.MM.dd."},
        "review_grade" : {"type":"long"},
        "review_mall"  : {"type":"keyword"},
        "review_title" : {"type":"text","analyzer":"korean"},
        "review_id"    : {"type":"keyword"},
        "review_accuracy" : {"type":"integer"},
        "review_tag" : {"type":"keyword"},
    }
}

es.indices.put_mapping("review",body=json.dumps(review_mapping),index=review_index_name)

In [ ]:
# check the mapping
print(es.indices.get_mapping(item_index_name,"item"))
print("-------------------------------------")
print(es.indices.get_mapping(review_index_name,"review"))

#### 5. put data into index

In [17]:
def generate_action(_index,_type):
    def _generate_action(_id, _source):
        return {
            "_index"  : _index,
            "_type"   : _type,
            "_id"     : _id,
            "_source" : _source
        }
    return _generate_action

**item document 넣기**

In [18]:
action = generate_action(_index=item_index_name,_type="item")
actions = [action(row['nv_mid'],row) for row in item_df.to_dict(orient='records')]

In [19]:
# Bulk helper를 이용하면 훨씬 더 빨리 넣을 수 있음
start = time.time()
helpers.bulk(es,actions,stats_only=False,chunk_size=2000,raise_on_error=False)
end = time.time()
print("consumed time --- {}".format(end-start));

consumed time --- 13.75279951095581


**review document 넣기**

In [ ]:
sample = review_df[review_df.review_date >="2017.06.01."]

In [ ]:
action = generate_action(_index=review_index_name,_type="review")
actions = [action(row['review_id'],row) for row in sample.to_dict(orient='records')]

In [ ]:
# Bulk helper를 이용하면 훨씬 더 빨리 넣을 수 있음
start = time.time()
helpers.bulk(es,actions,stats_only=False,chunk_size=1000,raise_on_error=False)
end = time.time()
print("consumed time --- {}".format(end-start));

------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------

## 일래스틱서치의 Search Query 던지기

Reference : https://bakyeono.net/post/2016-08-20-elasticsearch-querydsl-basic.html

### 검색요청하기

**Match_all**
    
    모든 문서와 매치한다

In [ ]:
body = {
    "query" :{"match_all":{}}
}
es.search(index=review_index_name,body=json.dumps(body))

**match** : 옵션

    지정한 필드에 전문 검색을 수행한다.

In [ ]:
body = {
    "query" :{
        "match":{
            "brand_name":"데스커"
        }}
}
es.search(index=review_index_name,body=json.dumps(body))

### 출력 필드 선택

**stored_fields** : [필드,...]

    보고 싶은 필드만 선택해서 받아올 수 있다.(네트워크 비용 감소)

In [ ]:
# 구버전 : fields -> stored_fields로 변경되었음
body = {
    "stored_fields" : ["review"],
    "query" :{
        "match":{
            "brand_name":"데스커"
        }
    }
}
es.search(index=review_index_name,body=json.dumps(body))

**from**: 값  |  **size**: 값
    
    from, size 옵션으로 검색 결과를 분할할 수 있다.
    예) 검색 결과 중 10번째 문서부터 5개의 문서 가져오기

In [ ]:
# 구버전 : fields -> stored_fields로 변경되었음
body = {
    "from":10,
    "size":5,
    "query" :{
        "match":{
            "brand_name":"데스커"
        }
    }
}
es.search(index=review_index_name,body=json.dumps(body))

**sort** : [{필드:  {옵션, ...}},...]
    
    sort 옵션으로 검색 결과를 정렬할 수 있다.

In [ ]:
body = {
    "sort":[
        {"review_accuracy" :{"order":"desc"}}
    ],
    "query": {
        "match" : {
            "brand_name":"데스커"
        }
    }
}
es.search(index=review_index_name,body=json.dumps(body))

### 쿼리와 필터의 구분
    
    쿼리와 필터는 둘 다 문서를 걸러내고 선택하는 용도이므로 비슷하지만, 구체적인 쓰임새가 다름.


| 쿼리 | 필터 |
|----|----|
|연관성| yes/no |
|캐시불가|캐시 가능|
|느림|빠름|

루씬은 아래와 같은 형태로 역 색인표를 만든다.

|필드|텀 |문서1|문서2|문서3|문서N|
|---|---|---|---|---|---|
|title|민주노총|1|0|0|...|
|title|한상균|0|1|0|...|
|title|편지|1|1|1|...|
|genre|편지|1|0|1|...|

캐시는 필터 전용 역색인표라고 할 수 있다. 마치 역색인표의 일부를 뽑아낸 것과 비슷한 모양으로 저장된다.

필터 종류, 필드, 텀에 의해 캐시의 키를 정하고, 필터의 결과를 비트벡터 형태로 저장해둔다. 예를 들어, 텀 필터의 결과는 다음과 같이 캐시된다.

필터는 Bool Query에 속하는 개념.

Bool Query type에는 
    
    - filter : filter 내 항목 모두를 밪아야 Okay 
    - must : must 내 항목 모두를 맞아야 Okay
    - should : should 내 항목 중에 하나라도 맞으면 Okay
    - must_not : must_not 내 항목 모두 없어야 Okay
    
    filter vs must : 점수를 계산하냐 안하냐의 차이

이전에는 filtered가 있었는데 deprecated 된 듯

In [ ]:
body = {
    "query": {
        "bool" : {
            "filter" : [
                { "range" : {
                    "review_grade" :{
                        "gte" :1,
                        "lte" :3
                        }
                    }
                },
                {"terms" : {
                    "brand_name":["데스커","한샘"]
                 }
                }
            ]            
        }
    }
}
es.search(index=review_index_name,body=json.dumps(body))

-----------------------------------
reference : https://www.elastic.co/blog/text-classification-made-easy-with-elasticsearch